In [ ]:
# 1. CÀI ĐẶT THƯ VIỆN
%pip install librosa soundfile numpy pandas scikit-learn matplotlib seaborn tqdm --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm

In [6]:
DATA_PATH = r"D:\DEEPFAKE_PROJECT\SER_Project\data\audio_speech_actors_01-24"

In [ ]:
# 2. TẢI DATAFRAME
emotion_dict = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}
def get_emotion_from_filename(filename):
    part = filename.split('-')[2]
    return emotion_dict.get(part, 'unknown')

data = []
for actor in os.listdir(DATA_PATH):
    if actor.startswith("Actor_"):
        actor_path = os.path.join(DATA_PATH, actor)
        for file in os.listdir(actor_path):
            if file.endswith(".wav"):
                emotion = get_emotion_from_filename(file)
                file_path = os.path.join(actor_path, file)
                data.append([file_path, emotion])

df = pd.DataFrame(data, columns=['path', 'emotion'])
print("Tổng số file:", len(df))
df.head()

Tổng số file: 1440


,path,emotion
0,D:\DEEPFAKE_PROJECT\SER_Project\data\audio_spe...,neutral
1,D:\DEEPFAKE_PROJECT\SER_Project\data\audio_spe...,neutral
2,D:\DEEPFAKE_PROJECT\SER_Project\data\audio_spe...,neutral
3,D:\DEEPFAKE_PROJECT\SER_Project\data\audio_spe...,neutral
4,D:\DEEPFAKE_PROJECT\SER_Project\data\audio_spe...,calm


In [ ]:
# 3. HÀM TRÍCH XUẤT ĐẶC TRƯNG ÂM THANH
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None, res_type='kaiser_fast')
    
    # MFCCs
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    
    # Chroma
    stft = np.abs(librosa.stft(y))
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
    
    # Spectral Contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sr).T, axis=0)
    
    # Gộp tất cả đặc trưng lại
    features = np.hstack([mfccs, chroma, contrast])
    return features

In [9]:
X, Y = [], []

for index, row in tqdm(df.iterrows(), total=len(df)):
    file_path = row['path']
    emotion = row['emotion']
    try:
        features = extract_features(file_path)
        X.append(features)
        Y.append(emotion)
    except Exception as e:
        print(f"Lỗi khi xử lý file {file_path}: {e}")

print("Hoàn tất trích xuất đặc trưng.")

100%|██████████| 1440/1440 [01:46<00:00, 13.47it/s]

Hoàn tất trích xuất đặc trưng.


In [10]:
X = np.array(X)
features_df = pd.DataFrame(X)
features_df['emotion'] = Y

print("Kích thước ma trận đặc trưng:", features_df.shape)
features_df.head()

Kích thước ma trận đặc trưng: (1440, 60)


,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,emotion
0,-726.217224,68.541420,3.293398,12.205300,5.510278,13.667408,-2.983828,3.098029,-3.310813,-1.564384,...,0.759382,0.733590,14.859834,8.485176,11.536135,13.140823,14.896630,15.938653,17.161146,neutral
1,-719.128296,70.201569,1.168397,13.122541,7.836950,14.411290,-4.111360,4.468973,-3.539367,-3.658607,...,0.772159,0.744746,15.131423,9.101594,12.306145,13.242768,14.797068,16.028111,17.303416,neutral
2,-714.995728,69.689346,3.924564,11.924190,6.421723,11.011614,-2.878103,4.509558,-4.476109,-2.671549,...,0.719798,0.723376,14.471466,9.088758,12.077312,13.564629,15.356175,16.092042,17.107516,neutral
3,-710.975281,67.564880,5.782241,13.230726,6.190845,12.628252,-1.675169,5.657494,-4.950634,-3.477545,...,0.742405,0.765174,16.239611,8.943198,12.414027,14.054614,15.625618,15.486327,17.372365,neutral
4,-759.921753,75.783524,6.023605,14.557394,6.454187,14.631508,-3.004551,4.620970,-5.200016,-0.707430,...,0.775044,0.727207,15.227692,9.126893,12.107234,13.334177,15.277864,15.372324,16.627257,calm


In [11]:
# 6. LƯU FILE ĐẶC TRƯNG RA CSV
save_path = "ravdess_features.csv"
features_df.to_csv(save_path, index=False)
print(f"Đã lưu đặc trưng ra file: {save_path}")

Đã lưu đặc trưng ra file: ravdess_features.csv
